![](https://www.broadinstitute.org/files/images/GoogleGenomics.png)

## GenX secondary analysis on GCP - GATK4

![](https://software.broadinstitute.org/gatk/img/pipeline_overview.png)

In [ ]:
%%bash
git clone https://github.com/gatk-workflows/broad-prod-wgs-germline-snps-indels.git  #to run it on GCP
git clone https://github.com/openwdl/wdl.git open_wdl               

## In order to run the GATK pipeline, we need our pipeline yaml file, wdl file, inputs, and options.
https://software.broadinstitute.org/gatk/best-practices/workflow?id=11145

In [1]:
%%bash
cat broad-prod-wgs-germline-snps-indels/README.md

# prod-wgs-germline-snps-indels
### Purpose : 
Workflows used in production at Broad for germline short variant discovery in WGS data. 

### PairedSingleSampleWF :
This WDL pipeline implements data pre-processing and initial variant calling (GVCF
generation) according to the GATK Best Practices (June 2016) for germline SNP and
Indel discovery in human whole-genome sequencing (WGS) data.

#### Requirements/expectations
- Human whole-genome pair-end sequencing data in unmapped BAM (uBAM) format
- One or more read groups, one per uBAM file, all belonging to a single sample (SM)
- Input uBAM files must additionally comply with the following requirements:
- - filenames all have the same suffix (we use ".unmapped.bam")
- - files must pass validation by ValidateSamFile
- - reads are provided in query-sorted order
- - all reads must have an RG tag
- Reference genome must be Hg38 with ALT contigs
#### Outputs 
- Cram, cram index, and cram md5 
- GVCF and its gvcf index 
- BQSR Report
- Several 

#### Let's see what the input (reference and test genomes) looks like

In [13]:
%%bash
cat broad-prod-wgs-germline-snps-indels/PairedEndSingleSampleWf.hg38.inputs.json

{
  "##_COMMENT1": "Take note of the .64 extensions on the reference files, issues between 32 and 64 bit OS",

  "##_COMMENT2": "SAMPLE NAME AND UNMAPPED BAMS - read the README to find other examples.",
  "PairedEndSingleSampleWorkflow.sample_name": "NA12878",
  "PairedEndSingleSampleWorkflow.base_file_name": "NA12878",
  "PairedEndSingleSampleWorkflow.flowcell_unmapped_bams": ["gs://broad-public-datasets/NA12878_downsampled_for_testing/unmapped/H06HDADXX130110.1.ATCACGAT.20k_reads.bam",
    "gs://broad-public-datasets/NA12878_downsampled_for_testing/unmapped/H06HDADXX130110.2.ATCACGAT.20k_reads.bam",
    "gs://broad-public-datasets/NA12878_downsampled_for_testing/unmapped/H06JUADXX130110.1.ATCACGAT.20k_reads.bam"],
  "PairedEndSingleSampleWorkflow.final_gvcf_base_name": "NA12878",
  "PairedEndSingleSampleWorkflow.unmapped_bam_suffix": ".bam",

  "##_COMMENT3": "REFERENCES",
  "PairedEndSingleSampleWorkflow.fingerprint_genotypes_file": "gs://dsde-data-na12878-public/NA12878.hg38.refere

#### Lets take a look at the  yaml config file

In [11]:
%%bash
cat open_wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \

name: WDL Runner
description: Run a workflow defined by a WDL file

inputParameters:
- name: WDL
  description: Workflow definition
- name: WORKFLOW_INPUTS
  description: Workflow inputs
- name: WORKFLOW_OPTIONS
  description: Workflow options

- name: WORKSPACE
  description: Cloud Storage path for intermediate files
- name: OUTPUTS
  description: Cloud Storage path for output files

docker:
  imageName: gcr.io/broad-dsde-outreach/wdl_runner:2017_10_02

  cmd: >
    /wdl_runner/wdl_runner.sh

resources:
  minimumRamGb: 3.75


#### What does our WDL file look like?

In [3]:
%%bash
cat broad-prod-wgs-germline-snps-indels/PairedEndSingleSampleWf.gatk4.0.wdl

## Copyright Broad Institute, 2017
##
## This WDL pipeline implements data pre-processing and initial variant calling (GVCF
## generation) according to the GATK Best Practices (June 2016) for germline SNP and
## Indel discovery in human whole-genome sequencing (WGS) data.
##
## Requirements/expectations :
## - Human whole-genome pair-end sequencing data in unmapped BAM (uBAM) format
## - One or more read groups, one per uBAM file, all belonging to a single sample (SM)
## - Input uBAM files must additionally comply with the following requirements:
## - - filenames all have the same suffix (we use ".unmapped.bam")
## - - files must pass validation by ValidateSamFile
## - - reads are provided in query-sorted order
## - - all reads must have an RG tag
## - GVCF output names must end in ".g.vcf.gz"
## - Reference genome must be Hg38 with ALT contigs
##
## Runtime parameters are optimized for Broad's Google Cloud Platform implementation.
## For program versions, see docker containers.
##
## 

#### Our options file looks like this

In [4]:
%%bash
cat broad-prod-wgs-germline-snps-indels/PairedEndSingleSampleWf.gatk4.0.options.json

{
  "read_from_cache":false,
  "default_runtime_attributes": {
    "zones": "us-central1-a us-central1-b us-east1-d us-central1-c us-central1-f us-east1-c",
    "docker": "broadinstitute/gatk:4.0.0.0"
  }
}


#### Create resources for outputs/logs etc. (if you don't have it)

In [5]:
%%bash
gsutil ls

gs://genomics-labs/


In [15]:
%env GCS=gs://genomics-labs/gatk4

env: GCS=gs://genomics-labs/gatk4


In [10]:
%env GATK4_DIR=/content/datalab/notebooks/Genomics/GATK/broad-prod-wgs-germline-snps-indels

env: GATK4_DIR=/content/datalab/notebooks/Genomics/GATK/broad-prod-wgs-germline-snps-indels


In [14]:
pwd

u'/content/datalab/notebooks/Genomics/GATK'

#### configure the pipeline and kick it off

In [26]:
%%bash

gcloud alpha genomics pipelines run \
  --pipeline-file /content/datalab/notebooks/Genomics/GATK/open_wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \
  --zones us-east1-c \
  --memory 5 \
  --logging $GCS/logging \
  --inputs-from-file WDL=$GATK4_DIR/PairedEndSingleSampleWf.gatk4.0.wdl \
  --inputs-from-file WORKFLOW_INPUTS=$GATK4_DIR/PairedEndSingleSampleWf.hg38.inputs.json \
  --inputs-from-file WORKFLOW_OPTIONS=$GATK4_DIR/PairedEndSingleSampleWf.gatk4.0.options.json \
  --inputs WORKSPACE=$GCS/workspace \
  --inputs OUTPUTS=$GCS/outputs

Running [operations/EK2d-qebLBirs8i0qaDnwEsglOj89t8GKg9wcm9kdWN0aW9uUXVldWU].


#### verify status of pipeline

In [59]:
%%bash 
gcloud alpha genomics operations describe ELanwaebLBi--dL-v9e_1MoBIJTo_PbfBioPcHJvZHVjdGlvblF1ZXVl --format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2018-02-20T20:38:44.374468118Z'
  - description: pulling-image
    startTime: '2018-02-20T20:38:44.374545767Z'
  - description: localizing-files
    startTime: '2018-02-20T20:39:19.251328689Z'
  - description: running-docker
    startTime: '2018-02-20T20:39:19.251380153Z'
  - description: delocalizing-files
    startTime: '2018-02-20T21:47:29.129516274Z'
  - description: ok
    startTime: '2018-02-20T21:47:30.451156310Z'


#### Verify pipeline stages in console and in the bucket. Outputs consist of:
- Cram, cram index, and cram md5 
- GVCF and its gvcf index 
- BQSR Report
- Several Summary Metrics 

In [16]:
%%bash 
gsutil ls $GCS

gs://genomics-labs/gatk4/logging/
gs://genomics-labs/gatk4/outputs/
gs://genomics-labs/gatk4/temp/
gs://genomics-labs/gatk4/workspace/


In [17]:
%%bash 
gsutil ls $GCS/outputs

gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.base_distribution_by_cycle.pdf
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.base_distribution_by_cycle_metrics
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.insert_size_histogram.pdf
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.insert_size_metrics
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_by_cycle.pdf
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_by_cycle_metrics
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_distribution.pdf
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_distribution_metrics
gs://genomics-labs/gatk4/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.unmapped.quality_yield_metrics
gs://genomics-labs/gatk4/outputs/H06HDADXX13011

In [1]:
%%bash
git clone https://github.com/gatk-workflows/gatk3-4-rnaseq-germline-snps-indels.git

Cloning into 'gatk3-4-rnaseq-germline-snps-indels'...


In [8]:
%env GCS=gs://genomics-labs/gatk4/rnaseq

env: GCS=gs://genomics-labs/gatk4/rnaseq


In [9]:
%env RNA_DIR=/content/datalab/notebooks/Genomics/GATK/gatk3-4-rnaseq-germline-snps-indels

env: RNA_DIR=/content/datalab/notebooks/Genomics/GATK/gatk3-4-rnaseq-germline-snps-indels


In [12]:
%%bash

gcloud alpha genomics pipelines run \
  --pipeline-file /content/datalab/notebooks/Genomics/GATK/open_wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \
  --zones us-east1-c \
  --memory 5 \
  --logging $GCS/logging \
  --inputs-from-file WDL=$RNA_DIR/rna-germline-variant-calling.wdl \
  --inputs-from-file WORKFLOW_INPUTS=$RNA_DIR/rna-germline-variant-calling.inputs.json \
  --inputs-from-file WORKFLOW_OPTIONS=$GATK4_DIR/PairedEndSingleSampleWf.gatk4.0.options.json \
  --inputs WORKSPACE=$GCS/workspace \
  --inputs OUTPUTS=$GCS/outputs

Running [operations/ELuz0sWvLBi02KeSwL2Ymy0glOj89t8GKg9wcm9kdWN0aW9uUXVldWU].


In [1]:
%%bash 
gcloud alpha genomics operations describe ELuz0sWvLBi02KeSwL2Ymy0glOj89t8GKg9wcm9kdWN0aW9uUXVldWU --format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2018-04-24T17:30:27.281378912Z'
  - description: pulling-image
    startTime: '2018-04-24T17:30:27.281446606Z'
  - description: localizing-files
    startTime: '2018-04-24T17:30:57.567922192Z'
  - description: running-docker
    startTime: '2018-04-24T17:30:57.567963138Z'
  - description: delocalizing-files
    startTime: '2018-04-24T22:24:59.525814164Z'
  - description: ok
    startTime: '2018-04-24T22:25:00.946193988Z'


In [7]:
%%bash
git clone https://github.com/gatk-workflows/gatk4-somatic-snvs-indels.git

Cloning into 'gatk4-somatic-snvs-indels'...


In [8]:
%env GCS=gs://genomics-labs/gatk4/somatic

env: GCS=gs://genomics-labs/gatk4/somatic


In [11]:
%env SOMATIC_DIR=/content/datalab/notebooks/Genomics/GATK/gatk4-somatic-snvs-indels

env: SOMATIC_DIR=/content/datalab/notebooks/Genomics/GATK/gatk4-somatic-snvs-indels


In [12]:
%%bash

gcloud alpha genomics pipelines run \
  --pipeline-file /content/datalab/notebooks/Genomics/GATK/open_wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \
  --zones us-east1-c \
  --memory 7 \
  --logging $GCS/logging \
  --inputs-from-file WDL=$SOMATIC_DIR/mutect2.wdl \
  --inputs-from-file WORKFLOW_INPUTS=$SOMATIC_DIR/mutect2.exome.inputs.json \
  --inputs-from-file WORKFLOW_OPTIONS=$GATK4_DIR/PairedEndSingleSampleWf.gatk4.0.options.json \
  --inputs WORKSPACE=$GCS/workspace \
  --inputs OUTPUTS=$GCS/outputs

Running [operations/ELv3pPSvLBiEse_zsIWf9w0glOj89t8GKg9wcm9kdWN0aW9uUXVldWU].


![](https://tctechcrunch2011.files.wordpress.com/2017/01/verily.jpg?w=738)

## GenX secondary analysis on GCP - DeepVariant 

In [45]:
cd /content/datalab/notebooks/Genomics/GATK/

/content/datalab/notebooks/Genomics/GATK


In [51]:
%%bash
chmod u+x vdv.sh
cat vdv.sh

#!/bin/bash
set -euo pipefail
# Set common settings.
PROJECT_ID=genomics-labs
OUTPUT_BUCKET=gs://genomics-labs/DV
STAGING_FOLDER_NAME=stage$random
OUTPUT_FILE_NAME=output.vcf
# Model for calling whole genome sequencing data.
MODEL=gs://deepvariant/models/DeepVariant/0.5.0/DeepVariant-inception_v3-0.5.0+cl-182548131.data-wgs_standard
# Model for calling exome sequencing data.
# MODEL=gs://deepvariant/models/DeepVariant/0.5.0/DeepVariant-inception_v3-0.5.0+cl-181413382.data-wes_standard
IMAGE_VERSION=0.5.1
DOCKER_IMAGE=gcr.io/deepvariant-docker/deepvariant:"${IMAGE_VERSION}"
DOCKER_IMAGE_GPU=gcr.io/deepvariant-docker/deepvariant_gpu:"${IMAGE_VERSION}"

# Run the pipeline.
gcloud alpha genomics pipelines run \
  --project "${PROJECT_ID}" \
  --pipeline-file deepvariant_pipeline.yaml \
  --logging "${OUTPUT_BUCKET}"/runner_logs \
  --zones us-west1-b \
  --inputs `echo \
      PROJECT_ID="${PROJECT_ID}", \
      OUTPUT_BUCKET="${OUTPUT_BUCKET}", \
      MODEL="${MODEL}", \
      DOCKER_IMA

In [48]:
%%bash
cat deepvariant_pipeline.yaml

name: deepvariant_pipeline
inputParameters:
- name: PROJECT_ID
- name: OUTPUT_BUCKET
- name: MODEL
- name: DOCKER_IMAGE
- name: DOCKER_IMAGE_GPU
- name: STAGING_FOLDER_NAME
- name: OUTPUT_FILE_NAME
docker:
  imageName: gcr.io/deepvariant-docker/deepvariant_runner
  cmd: |
    ./opt/deepvariant_runner/bin/gcp_deepvariant_runner \
      --project "${PROJECT_ID}" \
      --zones us-west1-b us-east1-d \
      --docker_image "${DOCKER_IMAGE}" \
      --docker_image_gpu "${DOCKER_IMAGE_GPU}" \
      --gpu \
      --outfile "${OUTPUT_BUCKET}"/"${OUTPUT_FILE_NAME}" \
      --staging "${OUTPUT_BUCKET}"/"${STAGING_FOLDER_NAME}" \
      --model "${MODEL}" \
      --bam gs://deepvariant/performance-testdata/HG002_NIST_150bp_downsampled_30x.bam \
      --ref gs://deepvariant/performance-testdata/hs37d5.fa.gz \
      --shards 1024 \
      --make_examples_workers 16 \
      --make_examples_cores_per_worker 64 \
      --make_examples_ram_per_worker_gb 240 \
      --make_examples_disk_per_worker_gb 200

In [53]:
%%bash
./vdv.sh

Running [operations/EKr5pqmbLBiPuZyKls7ViugBIJTo_PbfBioPcHJvZHVjdGlvblF1ZXVl].


In [1]:
%%bash 
gcloud alpha genomics operations describe EKr5pqmbLBiPuZyKls7ViugBIJTo_PbfBioPcHJvZHVjdGlvblF1ZXVl  --format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2018-02-20T21:41:30.806641101Z'
  - description: pulling-image
    startTime: '2018-02-20T21:41:30.806683911Z'
  - description: localizing-files
    startTime: '2018-02-20T21:41:53.236795357Z'
  - description: running-docker
    startTime: '2018-02-20T21:41:53.236827961Z'
  - description: delocalizing-files
    startTime: '2018-02-20T23:52:27.673265072Z'
  - description: ok
    startTime: '2018-02-20T23:52:29.096456825Z'


In [ ]:
%%bash 
gcloud alpha genomics operations cancel ELStlJCOLBiYuoPUosKa1wEg4r3yw4IMKg9wcm9kdWN0aW9uUXVldWU  --format='yaml(done, error, metadata.events)'

In [1]:
%%bash
gsutil ls gs://genomics-labs/DV

gs://genomics-labs/DV/output.vcf
gs://genomics-labs/DV/runner_logs/
gs://genomics-labs/DV/stage16214/


# GWAS

## Genomics Tertiary analysis on GCP

In [3]:
import datalab.bigquery as bq
variants_ds = bq.Dataset('GATK_Analysis')
variants_ds.create()


Dataset genomics-labs:GATK_Analysis

In [5]:
%%bash
pwd

/content/datalab/notebooks/Genomics/GATK


#### We now export variants from GATK4 outputs bucket and stream into big query

In [33]:
%%writefile vcf_to_bigquery.yaml
name: vcf-to-bigquery-pipeline 
docker: 
  imageName: gcr.io/gcp-variant-transforms/gcp-variant-transforms 
  cmd: | 
      ./opt/gcp_variant_transforms/bin/vcf_to_bq \
      --project genomics-labs \
      --input_pattern gs://genomics-labs/gatk4/outputs/*.vcf.gz \
      --output_table genomics-labs:GATK_Analysis.GATKtable \
      --staging_location gs://genomics-labs/gatk4/staging \
      --temp_location gs://genomics-labs/gatk4/temp \
      --job_name vcf-to-bigquery \
      --variant_merge_strategy MOVE_TO_CALLS \
      --runner DataflowRunner

Overwriting vcf_to_bigquery.yaml


In [38]:
%%bash
gcloud alpha genomics pipelines run \
    --project genomics-labs \
    --pipeline-file vcf_to_bigquery.yaml \
    --logging gs://genomics-labs/gatk4/temp/runner_logs \
    --zones us-east1-c \
    --service-account-scopes https://www.googleapis.com/auth/bigquery

Running [operations/ELny6IecLBjIwo-t8cPH_84BIJTo_PbfBioPcHJvZHVjdGlvblF1ZXVl].


In [46]:
%%bash
gcloud alpha genomics operations describe EIfH5oecLBj2wsKrmK6B9n4glOj89t8GKg9wcm9kdWN0aW9uUXVldWU  --format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2018-02-23T04:44:20.747665848Z'
  - description: pulling-image
    startTime: '2018-02-23T04:44:20.748311200Z'
  - description: localizing-files
    startTime: '2018-02-23T04:45:17.416764804Z'
  - description: running-docker
    startTime: '2018-02-23T04:45:17.416798125Z'
  - description: delocalizing-files
    startTime: '2018-02-23T04:53:51.101151908Z'
  - description: ok
    startTime: '2018-02-23T04:53:52.474059467Z'


In [47]:
variants_table = bq.Table('genomics-labs:GATK_Analysis.GATKtable')
variants_table.schema

[{ 'name': 'reference_name', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Reference name.' },
 { 'name': 'start_position', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'Start position (0-based). Corresponds to the first base of the string of reference bases.' },
 { 'name': 'end_position', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'End position (0-based). Corresponds to the first base after the last base in the reference allele.' },
 { 'name': 'reference_bases', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Reference bases.' },
 { 'name': 'alternate_bases', 'type': 'RECORD', 'mode':'REPEATED', 'description': 'One record for each alternate base (if any).' },
 { 'name': 'alternate_bases.alt', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Alternate base.' },
 { 'name': 'alternate_bases.MLEAC', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'Maximum likelihood expectation (MLE) for the allele counts (not necessarily the same as the AC), for each ALT allele, in the same order as listed' },
 { 'name': 'alternate_bases.MLEAF', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'Maximum likelihood expectation (MLE) for the allele frequency (not necessarily the same as the AF), for each ALT allele, in the same order as listed' },
 { 'name': 'names', 'type': 'STRING', 'mode':'REPEATED', 'description': 'Variant names (e.g. RefSNP ID).' },
 { 'name': 'quality', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'Phred-scaled quality score (-10log10 prob(call is wrong)). Higher values imply better quality.' },
 { 'name': 'filter', 'type': 'STRING', 'mode':'REPEATED', 'description': 'List of failed filters (if any) or "PASS" indicating the variant has passed all filters.' },
 { 'name': 'call', 'type': 'RECORD', 'mode':'REPEATED', 'description': 'One record for each call.' },
 { 'name': 'call.name', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Name of the call.' },
 { 'name': 'call.genotype', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'Genotype of the call. "-1" is used in cases where the genotype is not called.' },
 { 'name': 'call.phaseset', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Phaseset of the call (if any). "*" is used in cases where the genotype is phased, but no phase set ("PS" in FORMAT) was specified.' },
 { 'name': 'call.MIN_DP', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'Minimum DP observed within the GVCF block' },
 { 'name': 'call.AD', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'Allelic depths for the ref and alt alleles in the order listed' },
 { 'name': 'call.GQ', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'Genotype Quality' },
 { 'name': 'call.PID', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group' },
 { 'name': 'call.PGT', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another' },
 { 'name': 'call.PL', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification' },
 { 'name': 'call.SB', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'Per-sample component statistics which comprise the Fisher's Exact Test to detect strand bias.' },
 { 'name': 'call.DP', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'Approximate read depth (reads with MQ=255 or with bad mates are filtered)' },
 { 'name': 'ExcessHet', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'Phred-scaled p-value for exact test of excess heterozygosity' },
 { 'name': 'BaseQRankSum', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities' },
 { 'name': 'MQRankSum', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'Z-score From Wilcoxon rank sum test of A

#### Lets count the number of records (variant and reference segments) we have in the dataset and the total number of calls nested within those records

In [54]:
%%bq query

SELECT
  reference_name,
  COUNT(reference_name) AS num_records,
  SUM(ARRAY_LENGTH(call)) AS num_calls
FROM
  `genomics-labs.GATK_Analysis.GATKtable`
GROUP BY
  reference_name
ORDER BY
  reference_name

reference_name,num_records,num_calls
chr1,25280,25280
chr10,14651,14651
chr11,13933,13933
chr12,14009,14009
chr13,10490,10490
chr14,8754,8754
chr15,8532,8532
chr16,9685,9685
chr17,8554,8554
chr18,8420,8420


### Design

We'll be replicating a clinical study: Control Group and Test Group. 
One will be the reference and the other will the phenotype carrier (plug in here any pathology worth investigating at the Genetic level)

Our goal is to filter through BILLIONS of variants and pnpoint only the variant calls (for the sake of time  we'll do this within one chromosome, let's say the #12) that differs significantly between the case and control groups. If we can identify and isolate that we'll have a good chance of identifying a 'genenotype phenotype causality' (in other words identify what a the genetic level creates the pathology - in real life/research is not that easy but this is just a demo)

The case group for the purposes of this notebook will be individuals from the "EAS" (East Asian) super population. Variant data from the 1000 genomes dataset is publicly accessible within BigQuery.

We'll make the EAS group our test group and use the rest of the 1000 genomes as the control group.

We'll compare them and isolate the most meaningful differences

In [2]:
import datalab.bigquery as bq
variants_table = bq.Table('genomics-public-data:1000_genomes.variants')
variants_table.schema

[{ 'name': 'reference_name', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'An identifier from the reference genome or an angle-bracketed ID String pointing to a contig in the assembly file.' },
 { 'name': 'start', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'The reference position, with the first base having position 0.' },
 { 'name': 'end', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">' },
 { 'name': 'reference_bases', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Each base must be one of A,C,G,T,N (case insensitive). Multiple bases are permitted. The value in the POS field refers to the position of the first base in the String.' },
 { 'name': 'alternate_bases', 'type': 'STRING', 'mode':'REPEATED', 'description': 'List of alternate non-reference alleles called on at least one of the samples. ("at least one" not true for this dataset)' },
 { 'name': 'quality', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'phred-scaled quality score for the assertion made in ALT.' },
 { 'name': 'filter', 'type': 'STRING', 'mode':'REPEATED', 'description': 'PASS if this position has passed all filters, i.e. a call is made at this position. Otherwise, if the site has not passed all filters, a list of codes for filters that fail.' },
 { 'name': 'names', 'type': 'STRING', 'mode':'REPEATED', 'description': 'List of unique identifiers for the variant where available.' },
 { 'name': 'call', 'type': 'RECORD', 'mode':'REPEATED', 'description': 'Per-sample measurements.' },
 { 'name': 'call.call_set_id', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'The id of the callset from which this data was exported from the Google Genomics Variants API.' },
 { 'name': 'call.call_set_name', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'Sample identifier.' },
 { 'name': 'call.genotype', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'List of genotypes.' },
 { 'name': 'call.phaseset', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'If this value is null, the data is unphased.  Otherwise it is phased.' },
 { 'name': 'call.genotype_likelihood', 'type': 'FLOAT', 'mode':'REPEATED', 'description': 'List of genotype likelihoods.' },
 { 'name': 'call.DP', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'FORMAT=<ID=DP,Number=1,Type=Integer,Description="# high-quality bases">' },
 { 'name': 'call.DS', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'FORMAT=<ID=DS,Number=1,Type=Float,Description="Genotype dosage from MaCH/Thunder">' },
 { 'name': 'call.FT', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'None' },
 { 'name': 'call.GQ', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'FORMAT=<ID=GQ,Number=1,Type=Float,Description="Genotype quality">' },
 { 'name': 'call.PL', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'FORMAT=<ID=PL,Number=.,Type=Integer,Description="List of Phred-scaled genotype likelihoods, number of values is (#ALT+1)*(#ALT+2)/2">' },
 { 'name': 'call.SP', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'FORMAT=<ID=SP,Number=1,Type=Integer,Description="Phred-scaled strand bias P-value">' },
 { 'name': 'AA', 'type': 'STRING', 'mode':'NULLABLE', 'description': 'INFO=<ID=AA,Number=1,Type=String,Description="Ancestral Allele, ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/pilot_data/technical/reference/ancestral_alignments/README">' },
 { 'name': 'AC', 'type': 'INTEGER', 'mode':'REPEATED', 'description': 'INFO=<ID=AC,Number=.,Type=Integer,Description="Alternate Allele Count">' },
 { 'name': 'AC1', 'type': 'INTEGER', 'mode':'NULLABLE', 'description': 'INFO=<ID=AC1,Number=1,Type=Float,Description="Max-likelihood estimate of the first ALT allele count (no HWE assumption)">' },
 { 'name': 'AF', 'type': 'FLOAT', 'mode':'NULLABLE', 'description': 'INFO=<ID=AF,Number=1,Type=Float,Description="Global Allele Frequency based on AC/AN">' },
 { 'name': 'AF1', 'type': 'FLOAT', 'mode':'NULLA

### Classifying per-call variant positions into variant/non-variant groups

We can SUM the reference/alternate allele accounts for individual variant positions within chromosome 12.

The field call.genotype is an integer ranging from [-1, num_alternate_bases].

A value of negative one indicates that the genotype for the call is ambiguous (i.e., a no-call). A value of zero indicates that the genotype for the call is the same as the reference (i.e., non-variant). A value of one would indicate that the genotype for the call is the 1st value in the list of alternate bases (likewise for values >1).

In [3]:
%%sql --module allele_counts

SELECT 
    reference_name,
    start,
    reference_bases,
    alternate_bases,
    end,
    vt,    
    call.call_set_name AS call_set_name,
    # 1000 genomes phase 1 data is bi-allelic so there is only ever a single alt
    SUM(0 = call.genotype) WITHIN RECORD AS ref_count,
    SUM(1 = call.genotype) WITHIN RECORD AS alt_count,
FROM
  FLATTEN((
    SELECT
      reference_name,
      start,
      reference_bases,
      alternate_bases,
      end,
      vt,
      call.call_set_name,
      call.genotype,
    FROM
      $variants_table
    WHERE
      reference_name = '12' -- i.e., chromosome 12
    ),
    call)

In [4]:
bq.Query(allele_counts, variants_table=variants_table).sample().to_dataframe()

,reference_name,start,reference_bases,alternate_bases,end,vt,call_set_name,ref_count,alt_count
0,12,54319726,A,G,54319727,SNP,HG00261,0,2
1,12,54319726,A,G,54319727,SNP,HG00593,0,2
2,12,54319726,A,G,54319727,SNP,NA12749,1,1
3,12,54319726,A,G,54319727,SNP,HG00150,0,2
4,12,54319726,A,G,54319727,SNP,NA19675,0,2


### Assigning case and control groups

We can join our allele counts with metadata available in the sample info table.

Use this sample metadata to split the set of genomes into case and control groups based upon the super population group.

In [8]:
%%sql --module exp_groups

SELECT
  super_population,
  ('EAS' = super_population) AS is_case,
  call_set_name,
  reference_name,
  start,
  reference_bases,
  alternate_bases,
  end,
  vt,
  ref_count,
  alt_count,
FROM $allele_counts AS allele_counts
JOIN $sample_info_table AS samples
  ON allele_counts.call_set_name = samples.sample

In [ ]:
bq.Query(exp_groups, allele_counts=allele_counts,
                     sample_info_table=sample_info_table,
                     variants_table=variants_table).sample().to_dataframe()

In [0]:
%%sql
SELECT 
  vt,
  COUNT(*)
FROM $exp_groups
GROUP BY vt

### close to TWO BILLIONS SNPs. For the purposes of this DEMO we'll  limit the variants to only SNPs.

In [0]:
%%sql --module snps
SELECT * 
FROM $exp_groups
WHERE vt='SNP'

In [0]:
bq.Query(snps,
         exp_groups=exp_groups,
         allele_counts=allele_counts,
         sample_info_table=sample_info_table,
         variants_table=variants_table).sample().to_dataframe()

### SUMMING reference/alternate allele counts for case/control groups 

### Now that we've assigned each call set to either the case or the control group, we can tally up the counts of reference and alternate alleles within each of our assigned case/control groups, for each variant position, like so: 

In [0]:
%%sql --module grouped_counts

SELECT
    reference_name,
    start,
    end,
    reference_bases,
    alternate_bases,
    vt,
    SUM(ref_count + alt_count) AS allele_count,
    SUM(ref_count) AS ref_count,
    SUM(alt_count) AS alt_count,
    SUM(IF(TRUE = is_case, INTEGER(ref_count + alt_count), 0)) AS case_count,
    SUM(IF(FALSE = is_case, INTEGER(ref_count + alt_count), 0)) AS control_count,
    SUM(IF(TRUE = is_case, ref_count, 0)) AS case_ref_count,
    SUM(IF(TRUE = is_case, alt_count, 0)) AS case_alt_count,
    SUM(IF(FALSE = is_case, ref_count, 0)) AS control_ref_count,
    SUM(IF(FALSE = is_case, alt_count, 0)) AS control_alt_count,
FROM $snps
GROUP BY
    reference_name,
    start,
    end,
    reference_bases,
    alternate_bases,
    vt

In [0]:
bq.Query(grouped_counts,
         snps=snps,
         exp_groups=exp_groups,
         allele_counts=allele_counts,
         sample_info_table=sample_info_table,
         variants_table=variants_table).sample().to_dataframe()

### Quantify the statistical significance at each variant positions

### We can quantify the statistical significance of each variant position using the Chi-squared test. Furthermore, we can restrict our result set to only statistically significant variant positions for this experiment by ranking each position by its statistical signficance (decreasing) and thresholding the results for significance at p <= 5e-8 (chi-squared score >= 29.7).

In [0]:
%%sql --module results

SELECT
  reference_name,
  start,
  end,
  reference_bases,
  alternate_bases,
  vt,
  case_count,
  control_count,
  allele_count,
  ref_count,
  alt_count,
  case_ref_count,
  case_alt_count,
  control_ref_count,
  control_alt_count,


  ROUND(
    POW(ABS(case_ref_count - (ref_count/allele_count)*case_count) - 0.5,
      2)/((ref_count/allele_count)*case_count) +
    POW(ABS(control_ref_count - (ref_count/allele_count)*control_count) - 0.5,
      2)/((ref_count/allele_count)*control_count) +
    POW(ABS(case_alt_count - (alt_count/allele_count)*case_count) - 0.5,
      2)/((alt_count/allele_count)*case_count) +
    POW(ABS(control_alt_count - (alt_count/allele_count)*control_count) - 0.5,
      2)/((alt_count/allele_count)*control_count),
    3) AS chi_squared_score
FROM $grouped_counts
WHERE
  # For chi-squared, expected counts must be at least 5 for each group
  (ref_count/allele_count)*case_count >= 5.0
  AND (ref_count/allele_count)*control_count >= 5.0
  AND (alt_count/allele_count)*case_count >= 5.0
  AND (alt_count/allele_count)*control_count >= 5.0
HAVING
  # Chi-squared critical value for df=1, p-value=5*10^-8 is 29.71679
  chi_squared_score >= 29.71679
ORDER BY
  chi_squared_score DESC,
  allele_count DESC

In [0]:
bq.Query(results,
         grouped_counts=grouped_counts,
         snps=snps,
         exp_groups=exp_groups,
         allele_counts=allele_counts,
         sample_info_table=sample_info_table,
         variants_table=variants_table).sample().to_dataframe()

### the positions deemed significant do in fact have significantly different case/control counts for the alternate/reference bases.

### Computing Chi-squared statistics in BigQuery vs Python

### Let's compare these BigQuery-computed Chi-squared scores to ones calculated via Python's statistical packages

In [0]:
import numpy as np
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(np.array([ 
    [220, 352], # case 
    [1593, 19]  # control
]))

print 'Python Chi-sq score = %.3f' % chi2

### We can see that for the computation in Python the value matches 1086.505 from BigQuery. ( repeat this in R and see that result values match)

### Analyzing the GWAS results

### First, how many statistically significant variant positions did we find?

In [0]:
%%sql 
SELECT COUNT(*) AS num_significant_snps
FROM $results

### We now have a dataset that is sufficiently small to fit into memory on our instance, so let's pull the top 1000 SNP positions locally.

### Since we only need a subset of the columns, we can project our data first to remove unneeded columns.

In [0]:
%%sql --module sig_snps_dataset
SELECT * FROM (
  SELECT
    reference_name,
    start,
    reference_bases,
    alternate_bases,
    chi_squared_score
  FROM $results
  LIMIT 1000
)
ORDER BY start asc

In [0]:
sig_snps = bq.Query(sig_snps_dataset, 
                    results=results,
                    grouped_counts=grouped_counts,
                    snps=snps,
                    exp_groups=exp_groups,
                    allele_counts=allele_counts,
                    sample_info_table=sample_info_table,
                    variants_table=variants_table).to_dataframe()

sig_snps[:10]

### Let's visualize the distribution of significant SNPs along the length of the chromosome.

### The y-value of the charts indicates the Chi-squared score: larger values are more significant.

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

#g = sns.distplot(sig_snps['start'], rug=False, hist=False, kde_kws=dict(bw=0.1))
fig, ax = plt.subplots()
ax.scatter(sig_snps['start'], sig_snps['chi_squared_score'], alpha=0.3, c='red')
ax.set_ylabel('Chi-squared score')
ax.set_xlabel('SNP position (bp)')

### ok, now Let's zoom in on one region that contains a large number of very significant (high Chi Square Scores)  SNPs:

In [0]:
fig, ax = plt.subplots()
ax.scatter(sig_snps['start'], sig_snps['chi_squared_score'], alpha=0.5, c='red')
ax.set_xlim([3.3e7, 3.5e7])
ax.set_ylabel('Chi-squared score')
ax.set_xlabel('SNP position (bp)')

### Further exploration of statistically significant variant positions:

### We can take our analysis further by mapping selected variant positions back to the chromosome and visualizing call sets and reads. Let's retrieve 'The TOP ONE' SNP identified when ranked by the Chi-squared score:

In [0]:
%%sql --module top_snp
SELECT start
FROM $sig_snps_dataset
ORDER BY chi_squared_score desc
LIMIT 1

In [0]:
bq.Query(top_snp,
         sig_snps_dataset=sig_snps_dataset,
         results=results,
         grouped_counts=grouped_counts,
         snps=snps,
         exp_groups=exp_groups,
         allele_counts=allele_counts,
         sample_info_table=sample_info_table,
         variants_table=variants_table).results()

### That is IT. The most meaninful SNP variant beteween my control (read HEALTHY) group and my test (read SICK) group and this is 'the most likely culprit' (again, in reality is more complex but for the sake of the demo let's say this is our Agatha Christie moment and we pinpointed THE resonsible variant)    

### Now just to show  how the genome Browser looks let's the grab an arbitrary set of 10 callset IDs for rendering in the genome browser.

In [0]:
%%sql --module callset_ids
SELECT * FROM (
  SELECT call.call_set_id AS callset_id
  FROM $variants_table
  GROUP BY callset_id)
LIMIT 10

In [0]:
callsets_df = bq.Query(callset_ids, variants_table=variants_table).to_dataframe()
callsets = list(callsets_df['callset_id'])

In [0]:

from IPython.display import HTML
def gabrowse(dataset, reference_name, start_position, callset_ids):
    callsets_query_params = ''.join('&callsetId=%s&cBackend=GOOGLE' % callset_id for callset_id in callset_ids)
    url = ('https://gabrowse.appspot.com/#=&backend=GOOGLE&location=12%3A'
         + str(start_position)
         + callsets_query_params)
    return HTML('<iframe src="%s" width=1024 height=800></iframe>' % url)

### Now we can render the call sets and reads for the selected SNP position by embedding the GABrowse application directly in our notebook.

In [0]:
gabrowse('1000genomes', '12', 110571373, callsets)

### TO SUMMARIZE IT ALL: 
### FIRST: this notebook illustrated how to run TWO Secondary genomic analysis pipeline on GCP (VERILY + GATK)
### SECOND: how to conduct an entire GWAS experiment (which, for non-GCP users,  requires piping together 4 to 5 different tools and scripting the conections between the tools and the various languages et.c). 
### We accomplished all theee tasks from the same place (DATALAB) leaveraging many resources in the GCP Echosystem (GCS, GCE, GKE, BQ etc.) using publicly available VERY ALRGE variant data stored in BQ tables, and query it a AMAZING speed. Lastly after the filtering was done we imported a local copy of the most "clinically meaningful results" and visualized them with Python libraries. All from within one tool.


![](http://www.broadinstitute.org/gatk/img/GATK_BPP_white.gif)